In [ ]:
!pip install opencv-python

import cv2
import boto3
import json
import numpy as np
from IPython import display


# モデルのエンドポイント
CRAFT_ENDPOINT = "pytorch-inference-xxxx-xx-xx-xx-xx-xx-xxx"


def craft(client, image_bytes:bytes):
    # composite
    rawdata = image_bytes
    
    response = client.invoke_endpoint(
        EndpointName=CRAFT_ENDPOINT,
        ContentType="application/x-npy",
        Accept="application/json",
        Body=rawdata,
    )
    
    bboxes = json.loads(response["Body"].read())
    
    return bboxes


image = cv2.imread("sample2.png")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

_, encoded_image = cv2.imencode(".png", image)
image_bytes = encoded_image.tobytes()

client = boto3.client("sagemaker-runtime")

bboxes = craft(client, image_bytes)

for idx, bbox in enumerate(bboxes):
    cv2.rectangle(image, bbox[:2], bbox[2:], (0, 255, 0), 1)
    cv2.putText(image, str(idx), (bbox[0]+1, bbox[1]+1), cv2.FONT_HERSHEY_PLAIN, 0.8, (0, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(image, str(idx), bbox[:2], cv2.FONT_HERSHEY_PLAIN, 0.8, (0, 255, 0), 1, cv2.LINE_AA)

image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

_, image = cv2.imencode(".png", image)
image = display.Image(image)

display.display(image)